In [17]:
import pandas as pd

# Load the data
data = pd.read_excel('electronika_dataset.xlsx')

# Display the first few rows of the dataset to understand its structure
data.head()


,Order_ID,Email_new,Phone_new,Source,OrderDate,время,месяц,ChangeDate,DeliveryDate,PaymentDate,...,Quant,RowPrice,RowDiscount,RowSum,Brand,TN,TK,NomFullPath,Week,Nom_ID
0,1303000509_TT,55666668105117_iu29@yandex.ru,55485656-57565656575275,Онлайн-Резерв.,2016-03-30,08:46:30.000,201603,2016-03-30 09:31:57.000,2016-04-06 00:00:00.000,1900-01-01 00:00:00.000,...,1.0,0.0,0.0,0.0,NaN,NaN,NaN,Услуги/Доставка/,13.0,35554.0
1,1303000509_TT,55666668105117_iu29@yandex.ru,55485656-57565656575275,Онлайн-Резерв.,2016-03-30,08:46:30.000,201603,2016-03-30 09:31:57.000,2016-04-06 00:00:00.000,1900-01-01 00:00:00.000,...,1.0,3520.0,0.0,3520.0,NaN,NaN,NaN,Услуги/Страхование техники/Гарант +/,13.0,16686.0
2,1303000509_TT,55666668105117_iu29@yandex.ru,55485656-57565656575275,Онлайн-Резерв.,2016-03-30,08:46:30.000,201603,2016-03-30 09:31:57.000,2016-04-06 00:00:00.000,1900-01-01 00:00:00.000,...,1.0,35999.0,0.0,35999.0,Samsung,ТВ-Аудио,Телевизоры LCD,"Телевизоры, аудио, видео/Телевизоры/Smart теле...",13.0,95567.0
3,1303000510_TT,55666668105117_iu29@yandex.ru,55485656-57565656575275,Онлайн-Резерв.,2016-03-30,11:22:51.000,201603,2016-03-30 11:24:32.000,2016-04-06 00:00:00.000,1900-01-01 00:00:00.000,...,1.0,490.0,0.0,490.0,NaN,NaN,NaN,Услуги/Доставка/,13.0,29329.0
4,1303000510_TT,55666668105117_iu29@yandex.ru,55485656-57565656575275,Онлайн-Резерв.,2016-03-30,11:22:51.000,201603,2016-03-30 11:24:32.000,2016-04-06 00:00:00.000,1900-01-01 00:00:00.000,...,2.0,3990.0,0.0,7980.0,NaN,NaN,NaN,Установка и настройка техники/Установка и наст...,13.0,2748.0


In [18]:
# Assuming the dataset has columns: 'customer_id', 'product_id', and 'purchase_count'
# Group by customer_id and create a list of product_id sequences
customer_sequences = data.groupby('Phone_new')['TK'].apply(list).tolist()

# Display the first few sequences
print(customer_sequences[:5])


[[nan, 'Планшеты', nan, 'Телевизоры LCD', nan, 'Мобильные средства связи', nan, 'Телевизоры LCD', nan, 'Электропитание', nan, 'Силовая и садовая техника', nan, 'Мобильные средства связи', nan, 'Аксессуары для фото/видео', nan, 'Сетевое оборудование', nan, 'Автоэлектроника', nan, 'Силовая и садовая техника', nan, 'Приготовление пищи', nan, 'Автоэлектроника', 'Носители информации', nan, 'Мобильные средства связи', nan, 'Аксессуары для комп./ноутбуков/принтеров', nan, 'Аксессуары для комп./ноутбуков/принтеров', nan, 'Аксессуары для комп./ноутбуков/принтеров', nan, 'Аксессуары для комп./ноутбуков/принтеров', nan, 'Мобильные средства связи', nan, 'Мобильные средства связи', nan, 'Холодильное оборудование', nan, 'Товары для кухни', nan, 'Игрушки', nan, 'Красота и здоровье', nan, 'Приготовление пищи', nan, 'Телевизоры LCD', nan, 'Телевизоры LCD', nan, 'Телевизоры LCD', nan, 'Телевизоры LCD', nan, 'Аксессуары для смартфонов и планшетов', nan, 'Телевизоры LCD', nan, 'Телевизоры LCD', nan, 'Встр

In [19]:
from gensim.models import Word2Vec

# Define the Word2Vec model
model = Word2Vec(sentences=customer_sequences, vector_size=100, window=5, min_count=1, sg=1)

# Train the model
model.train(customer_sequences, total_examples=model.corpus_count, epochs=10)

# Save the model for future use
model.save("product_word2vec.model")

In [20]:
# Load the trained model
model = Word2Vec.load("product_word2vec.model")

# Function to find similar products
def recommend_similar_products(product_id, model, top_n=5):
    similar_products = model.wv.most_similar(positive=[product_id], topn=top_n)
    return [product for product, similarity in similar_products]

# Example usage
product_id = 'Телевизоры LCD' 
recommended_products = recommend_similar_products(product_id, model)
print(f"Products similar to {product_id}: {recommended_products}")


Products similar to Телевизоры LCD: ['Аксессуары ТВ-Аудио', 'Навигация', 'Домашние театры', 'Холодильное оборудование', 'Компьютеры портативные']


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec

# Load the dataset
data = pd.read_excel('electronika_dataset.xlsx')

# Assuming the dataset has columns: 'Phone_new', 'product_id', and 'purchase_count'
# Group by Phone_new and create a list of product_id sequences
customer_sequences = data.groupby('Phone_new')['TK'].apply(list).tolist()

# Split the sequences into training and test sets
train_sequences, test_sequences = train_test_split(customer_sequences, test_size=0.2, random_state=42)

# Display the first few sequences from the training set
print(train_sequences[:5])


[[nan, 'Фитнес'], [nan, 'Фитнес'], [nan, 'Периферийные устройства'], [nan, 'Мобильные средства связи'], [nan, 'Портативная электроника']]


In [22]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Prepare lists to hold true positives, false positives, and false negatives
true_positives = 0
false_positives = 0
false_negatives = 0
true_negatives = 0

# Iterate through test sequences and evaluate recommendations
for sequence in test_sequences:
    for product_id in sequence:
        actual_products = set(sequence)
        recommended_products = set(recommendations.get(product_id, []))
        
        true_positives += len(actual_products & recommended_products)
        false_positives += len(recommended_products - actual_products)
        false_negatives += len(actual_products - recommended_products)

# Calculate Precision, Recall, and F1 score
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

# Since we're working with individual product recommendations, calculating accuracy in the traditional sense is not applicable.
# Accuracy will be evaluated as the ratio of correctly recommended products to total products in the test set.
accuracy = true_positives / (true_positives + false_positives + false_negatives) if (true_positives + false_positives + false_negatives) > 0 else 0

# Print the evaluation metrics
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")


Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Accuracy: 0.0000
